<a href="https://colab.research.google.com/github/chiranjeet14/ML_Journey/blob/master/Hackerearth-Predict-the-genetic-disorders/5_genetic_testing_catBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import io
import gc
import time
from pprint import pprint
from datetime import date

# settings
import warnings
warnings.filterwarnings("ignore")
gc.enable()

In [ ]:
!pip3 install imbalanced-learn > /dev/null
!pip3 install catboost > /dev/null

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4101062452894924332, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7250706432
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10040602365090544973
 physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1"]

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gDrivePath = '/content/drive/MyDrive/Datasets/Hackerearth_genetic_testing/dataset/'

In [ ]:
df_train = pd.read_csv(gDrivePath+'train_preprocessed.csv')
df_test = pd.read_csv(gDrivePath+'test_preprocessed.csv')

In [ ]:
# df_train = pd.read_csv('train_preprocessed.csv')
# df_test = pd.read_csv('test_preprocessed.csv')

In [ ]:
df_train.sample(3)

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
4700,13.0,1.0,1.0,1.0,0.0,5.112789,41.000000,27.000000,1.0,2.0,2.0,2.000000,3.0,4.0,2.0,1.0,1.0,3.0,3.0,2.0,2.0,1.0,1.0,3.000000,1.000000,0.000000,0.000000,0.0,1.0,0.0,Mitochondrial genetic inheritance disorders,Mitochondrial myopathy
2012,10.0,1.0,0.0,0.0,0.0,4.708334,34.576361,41.913858,1.0,2.0,2.0,1.482048,3.0,1.0,1.0,2.0,2.0,4.0,0.0,1.0,1.0,0.0,2.0,3.655132,2.515345,0.621007,0.544312,1.0,0.0,1.0,Mitochondrial genetic inheritance disorders,Leigh syndrome
8842,1.0,0.0,0.0,0.0,1.0,4.837973,34.576335,41.916272,0.0,1.0,2.0,2.000000,3.0,2.0,3.0,2.0,2.0,4.0,3.0,2.0,2.0,1.0,2.0,3.185958,2.000000,1.000000,0.000000,1.0,0.0,0.0,Mitochondrial genetic inheritance disorders,Mitochondrial myopathy


In [ ]:
df_train['GeneticDisorder-DisorderSubclass'] = df_train['Genetic Disorder'] + '<->' + df_train['Disorder Subclass']

In [ ]:
df_train.sample(3)

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass,GeneticDisorder-DisorderSubclass
3875,6.962171,1.0,0.0,0.572433,0.0,4.993395,37.0,41.911178,0.0,1.0,1.0,1.0,3.000000,3.000000,3.0,2.0,1.505012,2.0,2.0,1.0,1.503392,0.0,1.493988,7.430034,4.0,1.0,0.0,1.0,0.0,0.0,Single-gene inheritance diseases,Tay-Sachs,Single-gene inheritance diseases<->Tay-Sachs
14873,4.000000,1.0,0.0,0.000000,0.0,4.677795,45.0,40.000000,0.0,2.0,1.0,2.0,1.000000,2.000000,3.0,2.0,1.000000,0.0,2.0,2.0,2.000000,4.0,2.000000,7.253566,2.0,0.0,1.0,1.0,1.0,1.0,Single-gene inheritance diseases,Cystic fibrosis,Single-gene inheritance diseases<->Cystic fibr...
12826,6.000000,0.0,0.0,0.000000,1.0,4.882556,50.0,31.000000,0.0,1.0,1.0,1.0,2.029911,2.509598,3.0,1.0,2.000000,3.0,4.0,2.0,1.000000,2.0,1.000000,9.164772,3.0,0.0,1.0,1.0,1.0,0.0,Mitochondrial genetic inheritance disorders,Leigh syndrome,Mitochondrial genetic inheritance disorders<->...


In [ ]:
# df_train[['Col1','Col2']] = df_train['GeneticDisorder-DisorderSubclass'].str.split("<->",expand=True)

In [ ]:
# df_train.sample(3)

### Checking if the dataset is balanced/imbalanced

In [ ]:
target_count = df_train['GeneticDisorder-DisorderSubclass'].value_counts()
target_count

Mitochondrial genetic inheritance disorders<->Leigh syndrome                         4683
Mitochondrial genetic inheritance disorders<->Mitochondrial myopathy                 3971
Single-gene inheritance diseases<->Cystic fibrosis                                   3145
Single-gene inheritance diseases<->Tay-Sachs                                         2556
Multifactorial genetic inheritance disorders<->Diabetes                              1653
Single-gene inheritance diseases<->Hemochromatosis                                   1228
Mitochondrial genetic inheritance disorders<->Leber's hereditary optic neuropathy     587
Multifactorial genetic inheritance disorders<->Alzheimer's                            133
Multifactorial genetic inheritance disorders<->Cancer                                  91
Name: GeneticDisorder-DisorderSubclass, dtype: int64

### Splitting Data into train-cv

In [ ]:
target_labels = df_train['GeneticDisorder-DisorderSubclass'].values

df_train.drop(['Genetic Disorder','Disorder Subclass', 'GeneticDisorder-DisorderSubclass'], axis=1, inplace=True)
df_test.drop(['Genetic Disorder','Disorder Subclass', 'GeneticDisorder-DisorderSubclass'], axis=1, inplace=True, errors='ignore')

In [ ]:
# classification split for genetic_disorder_labels
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_train, target_labels, test_size=0.1, random_state=50)

### Over Sampling using SMOTE

In [ ]:
# https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/
from imblearn.over_sampling import SMOTE
smote_overSampling = SMOTE()
X_train,y_train = smote_overSampling.fit_resample(X_train,y_train)
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{"Mitochondrial genetic inheritance disorders<->Leber's hereditary optic neuropathy": 4192,
 'Mitochondrial genetic inheritance disorders<->Leigh syndrome': 4192,
 'Mitochondrial genetic inheritance disorders<->Mitochondrial myopathy': 4192,
 "Multifactorial genetic inheritance disorders<->Alzheimer's": 4192,
 'Multifactorial genetic inheritance disorders<->Cancer': 4192,
 'Multifactorial genetic inheritance disorders<->Diabetes': 4192,
 'Single-gene inheritance diseases<->Cystic fibrosis': 4192,
 'Single-gene inheritance diseases<->Hemochromatosis': 4192,
 'Single-gene inheritance diseases<->Tay-Sachs': 4192}

### Scaling data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_cv_scaled = scaler.transform(X_cv)

X_test_scaled = scaler.transform(df_test)

# X_train_scaled

## Hyperparameter tuning

### Hyperparameter tuning Catboost

In [ ]:
%%time

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from catboost import CatBoostClassifier

CBC = CatBoostClassifier(loss_function='MultiClass', use_best_model=True, task_type="GPU")

# Best Params: {'random_strength': 0.5, 'max_ctr_complexity': 3, 'learning_rate': 0.1, 'l2_leaf_reg': 5, 'iterations': 500, 'depth': 10, 'border_count': 200, 'bagging_temperature': 0.03}

parameters = {
    'depth':[3,1,2,6,4,5,7,8,9,10],
    'iterations':[250,100,500,1000],
    'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3],
    'l2_leaf_reg':[3,1,5,10,100],
    'border_count':[32,5,10,20,50,100,200],
    'bagging_temperature':[0.03,0.09,0.25,0.75],
    'random_strength':[0.2,0.5,0.8],
    'max_ctr_complexity':[1,2,3,4,5]
}

Grid_CBC = RandomizedSearchCV(estimator=CBC, param_distributions=parameters, verbose=1, n_iter=200, scoring='f1_weighted', cv=5)
Grid_CBC.fit(X_train_scaled, y_train, eval_set=(X_cv_scaled, y_cv))

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Streaming output truncated to the last 5000 lines.
530:	learn: 0.1882584	test: 1.4567668	best: 1.2999961 (50)	total: 7.36s	remaining: 6.5s
531:	learn: 0.1876487	test: 1.4566612	best: 1.2999961 (50)	total: 7.37s	remaining: 6.48s
532:	learn: 0.1872654	test: 1.4565363	best: 1.2999961 (50)	total: 7.38s	remaining: 6.47s
533:	learn: 0.1866905	test: 1.4564083	best: 1.2999961 (50)	total: 7.39s	remaining: 6.45s
534:	learn: 0.1860962	test: 1.4572080	best: 1.2999961 (50)	total: 7.41s	remaining: 6.44s
535:	learn: 0.1857054	test: 1.4571397	best: 1.2999961 (50)	total: 7.42s	remaining: 6.42s
536:	learn: 0.1853962	test: 1.4568422	best: 1.2999961 (50)	total: 7.43s	remaining: 6.41s
537:	learn: 0.1850218	test: 1.4572410	best: 1.2999961 (50)	total: 7.45s	remaining: 6.4s
538:	learn: 0.1846498	test: 1.4571666	best: 1.2999961 (50)	total: 7.47s	remaining: 6.39s
539:	learn: 0.1843121	test: 1.4576763	best: 1.2999961 (50)	total: 7.48s	remaining: 6.37s
540:	learn: 0.1838049	test: 1.4579492	best: 1.2999961 (50)	to

[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed: 143.8min finished


0:	learn: 2.0436854	test: 2.1392071	best: 2.1392071 (0)	total: 90ms	remaining: 44.9s
1:	learn: 1.9333316	test: 2.0860741	best: 2.0860741 (1)	total: 177ms	remaining: 44.1s
2:	learn: 1.8437370	test: 2.0393459	best: 2.0393459 (2)	total: 264ms	remaining: 43.8s
3:	learn: 1.7672037	test: 1.9972969	best: 1.9972969 (3)	total: 361ms	remaining: 44.8s
4:	learn: 1.6959827	test: 1.9579215	best: 1.9579215 (4)	total: 456ms	remaining: 45.2s
5:	learn: 1.6311238	test: 1.9130082	best: 1.9130082 (5)	total: 535ms	remaining: 44.1s
6:	learn: 1.5786048	test: 1.8828590	best: 1.8828590 (6)	total: 633ms	remaining: 44.6s
7:	learn: 1.5269323	test: 1.8459900	best: 1.8459900 (7)	total: 713ms	remaining: 43.9s
8:	learn: 1.4840558	test: 1.8213471	best: 1.8213471 (8)	total: 800ms	remaining: 43.7s
9:	learn: 1.4430399	test: 1.7897833	best: 1.7897833 (9)	total: 891ms	remaining: 43.7s
10:	learn: 1.4065868	test: 1.7680230	best: 1.7680230 (10)	total: 977ms	remaining: 43.5s
11:	learn: 1.3727993	test: 1.7484610	best: 1.7484610 

In [ ]:
print("Best Params:", Grid_CBC.best_params_)

Best Params: {'random_strength': 0.5, 'max_ctr_complexity': 3, 'learning_rate': 0.1, 'l2_leaf_reg': 5, 'iterations': 500, 'depth': 10, 'border_count': 200, 'bagging_temperature': 0.03}


### Create predictions

In [ ]:
predictions_test = Grid_CBC.predict(X_test_scaled)
predictions_test

array([['Multifactorial genetic inheritance disorders<->Diabetes'],
       ['Single-gene inheritance diseases<->Tay-Sachs'],
       ['Mitochondrial genetic inheritance disorders<->Mitochondrial myopathy'],
       ...,
       ['Mitochondrial genetic inheritance disorders<->Mitochondrial myopathy'],
       ['Multifactorial genetic inheritance disorders<->Diabetes'],
       ["Mitochondrial genetic inheritance disorders<->Leber's hereditary optic neuropathy"]],
      dtype=object)

In [ ]:
# for catboost

predictions_tests = []
for x in predictions_test:
  predictions_tests.append(x[0])

## Create Submission file

In [ ]:
predictions_genetic_disorder_test = []
predictions_disorder_subclass_test = []

for myString in predictions_tests:
  genetic_disorder, disorder_subclass = myString.split('<->')
  predictions_genetic_disorder_test.append(genetic_disorder) 
  predictions_disorder_subclass_test.append(disorder_subclass)

print("length genetic_disorder list:", len(predictions_genetic_disorder_test))
print("length disorder_subclass list:", len(predictions_disorder_subclass_test))

length genetic_disorder list: 9465
length disorder_subclass list: 9465


In [ ]:
read = pd.read_csv(gDrivePath+'test.csv')
read.shape

(9465, 43)

In [ ]:
submission = pd.DataFrame({
        "Patient Id": read["Patient Id"],
        "Genetic Disorder": predictions_genetic_disorder_test,
        "Disorder Subclass": predictions_disorder_subclass_test,
    })

In [ ]:
submission.head()

,Patient Id,Genetic Disorder,Disorder Subclass
0,PID0x4175,Multifactorial genetic inheritance disorders,Diabetes
1,PID0x21f5,Single-gene inheritance diseases,Tay-Sachs
2,PID0x49b8,Mitochondrial genetic inheritance disorders,Mitochondrial myopathy
3,PID0x2d97,Mitochondrial genetic inheritance disorders,Leigh syndrome
4,PID0x58da,Single-gene inheritance diseases,Cystic fibrosis


In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.to_csv(gDrivePath+'submission.csv', index=False)